In [27]:
import pandas as pd

OPENSNP_API = 'https://opensnp.org/data/{user_id}'
users = pd.read_csv('/Users/david/caffeine_addiction_PRS/datasets/user_ids.tsv', sep='\t', comment='#')

In [28]:
users

,caffeine_addiction,file_link
0,1,11008.23andme.9131
1,0,10787.ancestry.8931
2,0,10469.23andme.8739
3,1,10420.23andme.8678
4,1,6192.23andme.4696
5,1,9974.ancestry.8238
6,1,9869.23andme.8128
7,0,8977.23andme.7322
8,0,1.23andme.9
9,1,8655.ancestry.7006


In [29]:
data = {}
for i in users.iterrows():
    user = i[1]['file_link']
    print('User: {user}'.format(user=user))
    table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)
    print(table.head())
    data[user] = table

User: 11008.23andme.9131


/var/folders/px/dh9c3vv16s1fhpfkq3b79ws80000gn/T/ipykernel_70702/2052517618.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)


             0  1       2   3
0  rs548049170  1   69869  TT
1    rs9283150  1  565508  AA
2  rs116587930  1  727841  GG
3    rs3131972  1  752721  AG
4   rs12184325  1  754105  CC
User: 10787.ancestry.8931


/var/folders/px/dh9c3vv16s1fhpfkq3b79ws80000gn/T/ipykernel_70702/2052517618.py:5: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)


             0           1         2        3        4
0         rsid  chromosome  position  allele1  allele2
1  rs369202065           1    569388        G        G
2  rs199476136           1    569400        T        T
3    rs3131972           1    752721        G        G
4  rs114525117           1    759036        G        G
User: 10469.23andme.8739


...

/var/folders/px/dh9c3vv16s1fhpfkq3b79ws80000gn/T/ipykernel_70702/2052517618.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)


             0  1       2   3
0   rs12564807  1  734462  AA
1    rs3131972  1  752721  GG
2  rs148828841  1  760998  CC
3   rs12124819  1  776546  AA
4  rs115093905  1  787173  --
User: 8466.ancestry.6829


/var/folders/px/dh9c3vv16s1fhpfkq3b79ws80000gn/T/ipykernel_70702/2052517618.py:5: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)


             0           1         2        3        4
0         rsid  chromosome  position  allele1  allele2
1  rs369202065           1    569388        G        G
2  rs199476136           1    569400        T        T
3    rs3131972           1    752721        G        G
4  rs114525117           1    759036        G        G
User: 7018.23andme.5429
             0  1       2   3
0  rs548049170  1   69869  TT
1   rs13328684  1   74792  --
2    rs9283150  1  565508  AA
3      i713426  1  726912  --
4  rs116587930  1  727841  GG


/var/folders/px/dh9c3vv16s1fhpfkq3b79ws80000gn/T/ipykernel_70702/2052517618.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(OPENSNP_API.format(user_id=user), comment='#', sep='\t', header=None)


In [35]:
print(hola.apparent_encoding)

None


In [39]:

dframe_ancestry = data['10787.ancestry.8931']
dframe_ancestry[dframe_ancestry.isna().any(axis=1)]


,0,1,2,3,4


In [22]:
print(magic.__dir__())

['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__path__']
